In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from collections import Counter
import math
from math import log
import pprint

In [2]:
# fileName = 'Users/zhangxiao/Desktop/数据集/Mnist/mnist_train.csv'

In [3]:
# def loadData(fileName):
#     rawdata = pd.read_csv(fileName, header=None)
#     data = rawdata.iloc[:, 1:]
#     label = rawdata.iloc[:, 0]
#     data = data.copy()
#     for column in list(data.columns):
#         data[column] = np.where(data[column] >= 128, 1, 0)
#     dataArr = data.values
#     labelArr = label.values

#     return dataArr, labelArr

In [4]:
def create_data():
    datasets = [['青年', '否', '否', '一般', '否'],
               ['青年', '否', '否', '好', '否'],
               ['青年', '是', '否', '好', '是'],
               ['青年', '是', '是', '一般', '是'],
               ['青年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '好', '否'],
               ['中年', '是', '是', '好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '好', '是'],
               ['老年', '是', '否', '好', '是'],
               ['老年', '是', '否', '非常好', '是'],
               ['老年', '否', '否', '一般', '否'],
               ]
    features = [u'年龄', u'有工作', u'有自己的房子', u'信贷情况', u'类别']
    # 返回数据集和每个维度的名称
    return datasets, features

In [5]:
datasets, features = create_data()

In [6]:
train_data = pd.DataFrame(datasets, columns=features)

In [7]:
train_data

,年龄,有工作,有自己的房子,信贷情况,类别
0,青年,否,否,一般,否
1,青年,否,否,好,否
2,青年,是,否,好,是
3,青年,是,是,一般,是
4,青年,否,否,一般,否
5,中年,否,否,一般,否
6,中年,否,否,好,否
7,中年,是,是,好,是
8,中年,否,是,非常好,是
9,中年,否,是,非常好,是


In [8]:
# 经验熵
def calc_ent(datasets):
    
    data_length = len(datasets)
    label_count = {}
    
    for i in range(data_length):
        label = datasets[i][-1]
        if label not in label_count:
            label_count[label] = 0
        label_count[label] += 1
        
    ent = -sum([(p / data_length) * log(p / data_length, 2)
                for p in label_count.values()])
    return ent
# def entropy(y):
#     """
#     Entropy of a label sequence
#     """
#     hist = np.bincount(y)
#     ps = hist / np.sum(hist)
#     return -np.sum([p * np.log2(p) for p in ps if p > 0])


# 经验条件熵
def cond_ent(datasets, axis):
    data_length = len(datasets)
    feature_sets = {}
    for i in range(data_length):
        feature = datasets[i][axis]
        if feature not in feature_sets:
            feature_sets[feature] = []
        feature_sets[feature].append(datasets[i])
    cond_ent = sum(
        [(len(p) / data_length) * calc_ent(p) for p in feature_sets.values()])
    return cond_ent


# 信息增益
def info_gain(ent, cond_ent):
    return ent - cond_ent


def info_gain_train(datasets):
    count = len(datasets[0]) - 1
    ent = calc_ent(datasets)
#     ent = entropy(datasets)
    best_feature = {}
    for feature_num in range(count): #计算各个特征的信息增益
        feature_info_gain = info_gain(ent, cond_ent(datasets, axis=feature_num))
        if feature_num not in best_feature:
            best_feature[feature_num] = []
        best_feature[feature_num].append(feature_info_gain)
        print('特征({}) - info_gain - {:.3f}'.format(features[feature_num], feature_info_gain))
    # 比较大小
    best_ = max(best_feature, key=best_feature.get)
    print(best_)
    print(best_feature[best_][0])
    return '特征({})的信息增益最大，选择为根节点特征'.format(features[best_])

In [9]:
info_gain_train(np.array(datasets))

特征(年龄) - info_gain - 0.083
特征(有工作) - info_gain - 0.324
特征(有自己的房子) - info_gain - 0.420
特征(信贷情况) - info_gain - 0.363
2
0.4199730940219749


'特征(有自己的房子)的信息增益最大，选择为根节点特征'

In [26]:
# 定义节点类 二叉树
class Node():
    def __init__(self, root=True, label=None, feature_name=None, feature=None):
        self.root = root
        self.label = label
        self.feature_name = feature_name
        self.feature = feature
        self.tree = {}
        self.result = {
            'label:': self.label,
            'feature': self.feature_name,
            'tree': self.tree
        }

    def __repr__(self):
        return '{}'.format(self.result)

    def add_node(self, val, node):
        self.tree[val] = node

    def predict(self, features):
        if self.root is True:
            return self.label
        return self.tree[features[self.feature]].predict(features)


class DTree():
    def __init__(self, epsilon=0.1):
        self.epsilon = epsilon
        self._tree = {}

    # 熵
    # @staticmethod
    def calc_ent(self, datasets):
        data_length = len(datasets)
        label_count = {}
        for i in range(data_length):
            label = datasets[i][-1]
            if label not in label_count:
                label_count[label] = 0
            label_count[label] += 1
        ent = -sum([(p / data_length) * log(p / data_length, 2)
                    for p in label_count.values()])
        return ent

    # 经验条件熵
    def cond_ent(self, datasets, axis=0):
        data_length = len(datasets)
        feature_sets = {}
        for i in range(data_length):
            feature = datasets[i][axis]
            if feature not in feature_sets:
                feature_sets[feature] = []
            feature_sets[feature].append(datasets[i])
        cond_ent = sum([(len(p) / data_length) * self.calc_ent(p)
                        for p in feature_sets.values()])
        return cond_ent

    # 信息增益
    def info_gain(self, ent, cond_ent):
        return ent - cond_ent

    def info_gain_train(self, datasets):
        count = len(datasets[0]) - 1
        ent = self.calc_ent(datasets)
    #     ent = entropy(datasets)
        best_feature = {}
        for feature_num in range(count): #计算各个特征的信息增益,axis参数顺序遍历各个特征
            feature_info_gain = self.info_gain(ent, self.cond_ent(datasets, axis=feature_num))
            if feature_num not in best_feature:
                best_feature[feature_num] = []
            best_feature[feature_num].append(feature_info_gain)
            print('特征({}) - info_gain - {:.3f}'.format(features[feature_num], feature_info_gain))
        # 比较大小
        best_ = max(best_feature, key=best_feature.get)
        print('特征({})的信息增益最大，选择为根节点特征 \n'.format(features[best_]))
        return best_, best_feature[best_][0]
    
    def train(self, train_data):
        """
        input:数据集D(DataFrame格式)，特征集A，阈值eta
        output:决策树T
        """
        _, y_train, features = train_data.iloc[:, :-1], train_data.iloc[:, -1], train_data.columns[:-1]
                                                                                           
        # 1,若D中实例属于同一类Ck，则T为单节点树，并将类Ck作为结点的类标记，返回T
        if len(y_train.value_counts()) == 1:
            return Node(root=True, label=y_train.iloc[0])

        # 2, 若A为空，则T为单节点树，将D中实例树最大的类Ck作为该节点的类标记，返回T
        if len(features) == 0:
            return Node(
                root=True,label=y_train.value_counts().sort_values(ascending=False).index[0])

        # 3,计算最大信息增益 同5.1,Ag为信息增益最大的特征
        max_feature, max_info_gain = self.info_gain_train(np.array(train_data))
        max_feature_name = features[max_feature]
        print(max_feature_name)

        # 4,Ag的信息增益小于阈值eta,则置T为单节点树，并将D中是实例数最大的类Ck作为该节点的类标记，返回T
        if max_info_gain < self.epsilon:
            return Node(
                root=True,
                label=y_train.value_counts().sort_values(
                    ascending=False).index[0])

        # 5,构建Ag子集
        node_tree = Node(
            root=False, feature_name=max_feature_name, feature=max_feature)

        feature_list = train_data[max_feature_name].value_counts().index
#         print(train_data[max_feature_name].value_counts())
#         print(feature_list)
        for f in feature_list:
            sub_train_df = train_data.loc[train_data[max_feature_name] ==f].drop([max_feature_name], axis=1)

            # 6, 递归生成树
            sub_tree = self.train(sub_train_df)
            node_tree.add_node(f, sub_tree)

        # pprint.pprint(node_tree.tree)
        return node_tree

    def fit(self, train_data):
        self._tree = self.train(train_data)
        return self._tree

    def predict(self, X_test):
        return self._tree.predict(X_test)

In [27]:
datasets, features = create_data()
data_df = pd.DataFrame(datasets, columns=features)
dt = DTree()
tree = dt.fit(data_df)

特征(年龄) - info_gain - 0.083
特征(有工作) - info_gain - 0.324
特征(有自己的房子) - info_gain - 0.420
特征(信贷情况) - info_gain - 0.363
特征(有自己的房子)的信息增益最大，选择为根节点特征 

有自己的房子
否    9
是    6
Name: 有自己的房子, dtype: int64
Index(['否', '是'], dtype='object')
特征(年龄) - info_gain - 0.252
特征(有工作) - info_gain - 0.918
特征(有自己的房子) - info_gain - 0.474
特征(有工作)的信息增益最大，选择为根节点特征 

有工作
否    6
是    3
Name: 有工作, dtype: int64
Index(['否', '是'], dtype='object')


In [25]:
tree

{'label:': None, 'feature': '有自己的房子', 'tree': {'否': {'label:': None, 'feature': '有工作', 'tree': {'否': {'label:': '否', 'feature': None, 'tree': {}}, '是': {'label:': '是', 'feature': None, 'tree': {}}}}, '是': {'label:': '是', 'feature': None, 'tree': {}}}}

In [19]:
dt.predict(['中年','否','是','一般'])

'是'